In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append("../likelihood/")

import PSR_counts as gc
from likelihood import ll

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. The expected number of PS counts

Here, we describe how we calculate the expected number of PS counts, given the model parameters.  These computations are implemented in the file `get_counts_inline.pyx` in the `likelihood/` subfolder.  First, we describe the computation for a general PS population, with density function $\rho(s,\ell,b)$, where $(s,\ell,b)$ are spatial coordinates with origin at the Earth, and luminosity function $dN/dL$.  Note that $\ell$ and $b$ are Galactic longitude and latitude, respectively, while $s$ is the distance from the Earth.  The expected number of counts in a given spatial bin (labeled by $i,j$) and flux bin (labelled by $k$) is given by the integral

$$
N_{i,j,k} = \omega_{i,j,k} \int_{\Delta \Omega_{i,j}} d\ell d b \rho(s,\ell,b) s^2 \int_{4 \pi s^2 S_k^\text{min}}^{4 \pi s^2 S_k^\text{max}} {dN \over dL} dL \,,
$$ 

where $\omega_{i,j,k}$ is the efficiency factor for pulsar-like PS detection in that bin.  Note that we have assumed that $\omega_{i,j,k}$ is constant within the bin, so that we may bring it outside of the integral.
 

In the code, for both the disk and the Bulge, we normalize both $\rho$ and $dN/dL$ such that the full integral over all of space and flux is equal to the total number of sources $N$ for that population.  The number of sources is one of the model parameters.  To work with this convention, we must know how to properly normalize $\rho$ and $dN/dL$ .

The normalization of $dN/dL$ is common to both the disk and Bulge.  Let 

$$
{dN \over dL} = {N_0 \over L^\beta}
$$

for $L$ in the range $[L_\text{min}, L_\text{max}]$ and be zero outside of this range.  We will normalize $dN/dL$ so that it integrates to unity, which implies that
 
$$
N_0 = {\beta - 1 \over L_\text{min}^{1-\beta} - L_\text{max}^{1 - \beta} } \,.
$$  


We discuss $\rho$ now for the disk and Bulge.  We will use the normalization that $\int d^3x \rho = N$.

Below, we take a few artibtrary choices to illustrate to functionality.

In [8]:
N_stars = 10.0 # Example, why not
omega_ijk = 1.0 # Example, why not
i=3 # Example, why not
j=3 # Example, why not
k=1 # Example, why not

Ns = 100 # Number of s bins in s integration
Nang = 2 # Number of angular bins in angular integration


# These are also adjustable 
Lmin=1.0e31
Lmax = 1.0e36

# Theta mask: this masks the inner \theta degrees when calculating the expected number of counts
theta_mask = 2.0 #degrees

## 1.1. Bulge sources 

We characterize the Bulge PS population by

$$
\rho_\text{bulge} = {C \over r^\alpha} \,,
$$

where $r$ is the distance from the Galactic Center, so long as $r < r_\text{cut}$ .  $\rho$ is equal to zero for larger values of $r$ .  To have to proper normalization, it is straightforward to verify that
 
$$
C = {(3 - \alpha) N_\text{bulge} \over 4 \pi r_\text{cut}^{3 - \alpha} } \,.
$$

In [11]:
alpha = 2.60
beta = 1.20
rcut = 3.0 #kpc

N_bulge = gc.Nbulge_ijk(i,j,k,N_stars,omega_ijk,alpha,beta,rcut,Lmin,Lmax,Ns,Nang,theta_mask) 
print N_bulge

0.000628541643001
0.000260378877394


## 1.2. Disk sources

The disk is modeled by the distribution 

$$
\rho_\text{disk} = C R^n e^{-R / \sigma} e^{-|z| / z_0} \,,
$$

and we find that in this case

$$
C = {N_\text{disk} \over 4 \pi z_0 \sigma^{n+2} \Gamma(n+2)} \,.
$$

In [12]:
n = 2.35
z0 = 0.7 #kpc
sigma = 1.528 #kpc
beta=1.20

smax = 40.0 #kpc , how far out to integrate to in the radial direction?

N_disk = gc.Ndisk_ijk(i,j,k,N_stars,omega_ijk,n,sigma,z0,beta,Lmin,Lmax,Ns,Nang,smax,theta_mask) 
print N_disk

0.000260378877394


Now, let's calculate the total number of total bulge and disk counts, for the prior.
Here, we integrate over fluxes from $1.8 \times 10^{-5}$ MeV cm$^{-2}$ s$^{-1}$ to $6.539 \times 10^{-4}$ MeV cm$^{-2}$ s$^{-1}$.

In [13]:
Nang_total = 100 #We use a larger number of angular bins here, since we are integrating over the full sky
N_bulge_total = gc.Nbulge_total(N_stars,alpha,beta,rcut,Lmin,Lmax,Ns,Nang_total,theta_mask)
N_disk_total = gc.Ndisk_total(N_stars,n,sigma,z0,beta,Lmin,Lmax,Ns,Nang_total,smax,theta_mask)
print N_bulge_total,N_disk_total

0.180678272082 0.352989832573


# 2. The likelihood function

The `GCE-2FIG` code package uses the likelihood function presented in [1705.00009](https://arxiv.org/pdf/1705.00009.pdf).  The likelihood is implemented in the file `likelihood.pyx` in the `likelihood/` subfolder.  The sky is spatially binned in a cartesian grid, and the data consists of the number of pulsar candidates detected in each bin.  The model parameters characterize the disk and Bulge point source populations, and as we scan over the model parameters we calculated the expected number of detected sources in each bin, utilizing the _Fermi_-provided efficiency function for detecting sources at a given flux and spatial position.  The likelihood is then given by the product over all pixels of the Poisson probabilities to observe the data counts given the predicted model counts.

As an option, we also incorporate the prior distribution in [1705.00009](https://arxiv.org/pdf/1705.00009.pdf).

Arguments for the likelihood function are as follows:

```
    PSR_data : data binned in [longitude, latitude, flux]
    omega_ijk : efficiency in same binning as data
    Nbulge : number of bulge PSRs
    Ndisk : number of disk PSRs
    alpha : inner slope of the bulge spatial profile
    n : cylindrical radial index of the disk profile
    sigma : exponential cutoff of disk radius [kpc]
    z0 : exponential cutoff of disk height [kpc]
    Lmax_disk : max luminosity of the disk [erg/s]
    Lmax_bulge : max luminosity of the bulge [erg/s]
    beta_disk : index for luminosity function of disk
    beta_bulge : index for luminosity function of bulge
    Nang : number of angular steps per bin for the likelihood integral
    smax_disk : maximum distance to integrate through the disk from Earth [kpc]
    theta_mask : angle from the GC to be masked
    use_prior : whether to use the prior
    Nang_prior : number of angular steps per bin for the prior integral
```

Here we provide an example of evaluating the likelihood at a specific parameter point.

In [18]:
# Load data and efficiency files created in Notebooks 1 and 2
data = np.load("../data/PSR_data.npy") + np.load("../data/PSR_data_3fgl.npy") 
omega_ijk = np.load("../data/omega_ijk_int.npy")

In [19]:
ll(data, omega_ijk, N_bulge, N_disk, alpha, n, sigma, z0, Lmax, Lmax, beta, beta)

-1568.1928432458642